In [1]:
import qiskit

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal import jigsaw

from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.bv import bv_circuit_cmap
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear

from PatchedMeasCal import non_standard_coupling_maps

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from PatchedMeasCal.edge_bfs import CouplingMapGraph

from functools import partial

import qiskit.tools.jupyter

import random

In [2]:
grid = Grid(2, 2)
circuit = bv_circuit_cmap('110', 4, grid)

In [3]:
circuit.draw()

┌───┐      ░      ┌───┐ ░ ┌───┐
q_0: ┤ H ├──────░──────┤ X ├─░─┤ H ├
     ├───┤      ░ ┌───┐└─┬─┘ ░ ├───┤
q_1: ┤ H ├──────░─┤ X ├──■───░─┤ H ├
     ├───┤      ░ └─┬─┘      ░ ├───┤
q_2: ┤ H ├──────░───┼────────░─┤ H ├
     ├───┤┌───┐ ░   │        ░ ├───┤
q_3: ┤ H ├┤ Z ├─░───■────────░─┤ H ├
     └───┘└───┘ ░            ░ └───┘
c: 4/═══════════════════════════════

In [4]:
circuit.draw()

┌───┐      ░            ░ ┌───┐┌─┐         
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├─────────
     ├───┤      ░   │        ░ ├───┤└╥┘┌─┐      
q_1: ┤ H ├──────░───┼────■───░─┤ H ├─╫─┤M├──────
     ├───┤      ░   │    │   ░ ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├──────░───┼────┼───░─┤ H ├─╫──╫─┤M├───
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐ ░ ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├┤ Z ├─░─┤ X ├┤ X ├─░─┤ H ├─╫──╫──╫─┤M├
     └───┘└───┘ ░ └───┘└───┘ ░ └───┘ ║  ║  ║ └╥┘
c: 4/════════════════════════════════╩══╩══╩══╩═
                                     0  1  2  3

In [11]:
grid = Grid(2, 2)

In [148]:
def bv_c(bv_str, n_qubits, backend):
    circ = qiskit.QuantumCircuit(n_qubits, n_qubits)
    
    for i in range(n_qubits):
        circ.h(i)
    circ.z(n_qubits - 1)
    circ.barrier()
    
    coupling_map = grid.configuration().coupling_map
    
    target = [int(i) for i in bv_string] + [1]
    current = [0] * (n_qubits - 1) + [1]
    
    cnot_chain = cnot_network(current, target, coupling_map, n_qubits)
    
    for i in cnot_chain:
        circ.cnot(*i)   
    
    circ.barrier()
    
    for i in range(n_qubits):
        circ.h(i)
    
    return circ

In [150]:
bv_c('101', 4, grid).draw()

[0, 0, 0, 1]
0 3
2 3
[0, 0, 1, 1]
0 2


┌───┐      ░      ┌───┐ ░ ┌───┐
q_0: ┤ H ├──────░──────┤ X ├─░─┤ H ├
     ├───┤      ░      └─┬─┘ ░ ├───┤
q_1: ┤ H ├──────░────────┼───░─┤ H ├
     ├───┤      ░ ┌───┐  │   ░ ├───┤
q_2: ┤ H ├──────░─┤ X ├──■───░─┤ H ├
     ├───┤┌───┐ ░ └─┬─┘      ░ ├───┤
q_3: ┤ H ├┤ Z ├─░───■────────░─┤ H ├
     └───┘└───┘ ░            ░ └───┘
c: 4/═══════════════════════════════

In [147]:
def cnot_network(initial_state, target_state, coupling_map, n_qubits):
    network = []

    initial_state = copy.deepcopy(initial_state)

    distance_map = cmap_djikstra(coupling_map, n_qubits)

    ct = 0
    while initial_state != target_state and ct < 5:
        ct += 1
        print(initial_state)
        mask = [i ^ j for i, j in zip(initial_state, target_state)]

        # Get longest path distance remaining in the stack
        shortest_path = [float('inf'), None]
        for i in range(n_qubits):
            if mask[i] == 1:
                for j in range(n_qubits):
                    if initial_state[j] == 1 and i != j:
                        #shortest_path = cmap_shortest_path(j, i, distance_map, coupling_map)
                        if shortest_path[0] > distance_map[i][j]:
                            shortest_path = [distance_map[i][j], [j, i]]
                            print(i, j)


        path = cmap_shortest_path(*shortest_path[1], distance_map, coupling_map)
        for i, j in zip(path[:-1], path[1:]):
            network.append([i, j])
            initial_state[j] ^= initial_state[i]

    return network

    
    

In [138]:
initial_state

[1, 1, 1, 0]

In [139]:
coupling_map

[[0, 1], [1, 0], [0, 2], [2, 0], [2, 3], [3, 2], [1, 3], [3, 1]]

In [140]:
network

[[3, 1], [1, 0], [0, 2], [1, 3]]

In [127]:
initial_state

[1, 1, 1, 0]

In [94]:
initial_state == target_state

True

In [91]:
network

[[3, 2], [2, 0]]

In [84]:
mask

[1, 0, 1, 0]

In [89]:
path

[3, 2, 0]

In [86]:
#shortest_paths.sort(reverse=True)
longest_path

[2, [0, 3]]

In [26]:
cmg

[[0, 1], [0, 2], [2, 3], [1, 3]]

In [28]:
coupling_map_uniq = []
for i in coupling_map:
    if i[::-1] not in coupling_map_uniq:
        coupling_map_uniq.append(i)

[[0, 1], [1, 0], [0, 2], [2, 0], [2, 3], [3, 2], [1, 3], [3, 1]]
[[0, 1], [0, 2], [2, 3], [1, 3]]


In [53]:
import copy
def cmap_djikstra(cmap, n_qubits, root = 0):
        
    distances = [{i:0} for i in range(n_qubits)]
    
    cmap = copy.deepcopy(cmap)
    traversed = [root]
    edges_used = []
    nodes_found = []
    
    for c in cmap:
        distances[c[0]][c[1]] = 1
    
    while len(cmap) > 0:
        for t in traversed:
            for c in cmap:
                if c[0] == t:
                    edges_used.append(c)
                    nodes_found.append(c[1])

                    # Join
                    distances_t = distances[t]
                    distances_e = distances[c[1]]
                    for d_t in distances_t:
                        if d_t not in distances_e:
                            distances_e[d_t] = distances_t[d_t] + 1
                        else:
                            distances_e[d_t] = min(distances_e[d_t], distances_t[d_t] + 1)
                            distances_t[d_t] = min(distances_e[d_t] + 1, distances_t[d_t])
        traversed = nodes_found
        nodes_found = []
        
        for e in edges_used:
            cmap.remove(e)
            cmap.remove(e[::-1])
        edges_used = []
                
            
    # Symmetric Cleanup
    for i, d in enumerate(distances):
        for j in range(n_qubits):
            if j not in d:
                d[j] = distances[j][i]
    return distances
        

In [128]:
def djikstra_tree(coupling_map, n_qubits, root = 0):
    traversed = []
    front_nodes = [root]
    
    tree_cmap = []
    coupling_map = copy.deepcopy(coupling_map)
 
    
    while len(traversed) < n_qubits:
        next_front = []
        for t in front_nodes:
            for c in coupling_map:
                if c[0] == t:
                    if c[1] not in traversed and c[1] not in front_nodes and c[1] not in next_front:
                        next_front.append(c[1])
                        tree_cmap.append(c)
                        tree_cmap.append(c[::-1])
                        coupling_map.remove(c)
                        coupling_map.remove(c[::-1])
                   
        traversed += front_nodes
        front_nodes = next_front

    return tree_cmap
    
    

In [121]:
djikstra_tree(coupling_map, n_qubits)

0 1 [] [0] []
1 3 [0] [1] []
3 2 [0, 1] [3] []
2 0 [0, 1, 3] [2] []


[[0, 1], [1, 0], [1, 3], [3, 1], [3, 2], [2, 3]]

In [61]:
# This would be much faster with a proper graph structure
def cmap_shortest_path(start, end, distance_map, cmap):
    distance = distance_map[start][end]
    path = [start]
    curr_node = start
    while curr_node != end:
        next_node = None
        for i in distance_map[curr_node]:
            if [curr_node, i] in cmap or [i, curr_node] in cmap:
                if distance_map[i][end] == distance_map[curr_node][end] - 1:
                    next_node = i
                    break  
        path.append(next_node)
        curr_node = next_node
    return path

In [62]:
cmap_shortest_path(0, 3, grid_map, coupling_map)

[0, 1, 3]

In [56]:
grid_map = cmap_djikstra(coupling_map, 4)

In [48]:
%debug

> /tmp/ipykernel_179769/2413471292.py(42)cmap_djikstra()
     40     for d in distances:
     41         for e in distances:
---> 42             if d not in distances[e]:
     43                 distances[e][d] = d
     44     return distances

ipdb> e
{0: 0, 1: 1, 2: 1}
ipdb> d
*** Newest frame
ipdb> distances
[{0: 0, 1: 1, 2: 1}, {1: 0, 0: 1, 3: 1, 2: 2}, {2: 0, 0: 1, 3: 1, 1: 2}, {3: 0, 2: 1, 1: 1, 0: 2}]
ipdb> q


In [19]:
dir(grid.configuration())

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_data',
 'backend_name',
 'backend_version',
 'basis_gates',
 'conditional',
 'coupling_map',
 'dynamic_reprate_enabled',
 'from_dict',
 'gates',
 'local',
 'max_experiments',
 'max_shots',
 'memory',
 'n_qubits',
 'num_qubits',
 'open_pulse',
 'simulator',
 'supported_instructions',
 'to_dict']

In [22]:
coupling_map

[[0, 1], [1, 0], [0, 2], [2, 0], [2, 3], [3, 2], [1, 3], [3, 1]]